In [3]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle
from collections import defaultdict
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import random
import sys
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import sklearn.preprocessing as pp

In [4]:
with open('data/artists.p', 'rb') as fp:
    artists = pickle.load(fp)
    
with open('data/genres.p', 'rb') as fp:
    genres = pickle.load(fp)

with open('data/genres_clean.p', 'rb') as fp:
    genres_clean = pickle.load(fp)
    
with open('data/artists_clean.p', 'rb') as fp:
    artists_clean = pickle.load(fp)

In [102]:
#len(genres)

In [103]:
#len(artists)

In [104]:
#sample_dict = dict(random.sample(artists.items(), 80000))

In [105]:
#df_genres = pd.DataFrame(columns=genres_clean)

In [106]:
#df_artists = pd.DataFrame(sample_dict.keys(), columns=['artist'])

In [107]:
#df = pd.concat([df_artists,df_genres])

In [109]:
#for col in df.columns[1:]:
#    df[col].values[:] = 0

In [110]:
#%%timeit

#for index in df.index:
#    if df.loc[index,'artist'] in sample_dict.keys():
#        band = df['artist'][index]
#       print(mvp_dict[band])
#       mvp_df.loc[index,mvp_df[mvp_dict[band]] == 0, mvp_dict[band]] = 1
#        df.loc[df["artist"] == band, sample_dict[band]] = 1

In [111]:
#df.index.rename('artist_id', inplace=True)

In [112]:
#array_cols = df.columns[1:]

#array_cols[:5]

In [113]:
#df_for_array = df[array_cols]

In [114]:
#df_sparsed = df_for_array.astype(pd.SparseDtype("int"))

In [115]:
#sparse_array = df_sparsed.to_numpy()

In [116]:
#data_sparse = sparse.csr_matrix(df_sparsed)

In [117]:
# calculate similarity between each row (artist x genre)
#similarities_sparse = cosine_similarity(data_sparse, dense_output=False)

In [118]:
# returns index (column position) of top n similarities in each row
def top_n_idx_sparse(matrix, n):
    '''Return index of top n values in each row of a sparse matrix'''
    top_n_idx = []
    for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
        n_row_pick = min(n, ri - le)
        top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]])
    return top_n_idx

#artist_artist_similar = top_n_idx_sparse(similarities_sparse, 11)

In [119]:
# transforms result from sparse matrix into a dict as {matrix_index: [user_1, user_2,..., user_5]}
#artist_similarity_dict = {}
#for idx, val in enumerate(artist_artist_similar):
#        artist_similarity_dict.update({idx: val.tolist()})

In [120]:
# gets actual user ids from data based on sparse matrix position index
#similar_artists_final = {}
#for artist, similar_artists in artist_similarity_dict.items():
#    idx = mvp_df.index[artist]
#    values = []
#    for value in similar_artists:
#        values.append(mvp_df.index[value])

#    similar_artists_final.update({idx: values})

In [ ]:
# transforms list of artists: [similar_artist1, similar_artist2] into list of artist: [artist1, artist2]
#similar_bands = {}
#for artist, similar_artists in similar_artists_final.items():
    # remove the artist itself from similar_artists (since cos_sim(artist_1, artist_1) is 1)
#    try:
#        del similar_artists[similar_artists.index(artist)]
#    except:
#        pass
     # get artist ids from list of similar artists.
#    artists_rec = mvp_df[(mvp_df.index.isin(similar_artists))]
#    if artists_rec.empty:
#        continue
#    artists_sample = artists_rec.sample(n=10, replace=True).index.values
#    similar_bands.update({artist: list(set(artists_sample))})

In [ ]:
# transform dictionary into list of tuples and save on DataFrame
#artist_sim_tup = [(artist, sim_artist) for artist, similar_bands in similar_bands.items() for sim_artist in similar_bands]
#artist_similarity_df = pd.DataFrame(artist_sim_tup, columns=['artist_id', 'similar_artist_id'])

In [ ]:
#rec_batch = pd.merge(artist_similarity_df, mvp_df['artist'], on='artist_id', how='left')

In [ ]:
#artist_map_dict = {}

#for idx,row in enumerate(df['artist']):
    #print(row)
#    artist_map_dict[idx] = row

In [121]:
#rec_batch['similar_artist'] = rec_batch['similar_artist_id']

In [122]:
#rec_batch['similar_artist'] = rec_batch['similar_artist'].map(artist_map_dict)

In [ ]:
#rec_batch = (rec_batch.set_index(['artist',rec_batch.groupby('artist').cumcount()])['similar_artist']
#        .unstack()
#        .add_prefix('similar artist #')
#        .reset_index())

In [123]:
#rec_batch.head(50)

In [ ]:
# mongod server connection
#mongodb+srv://medrich:<password>@metisengineering.ddwda.mongodb.net/myFirstDatabase?retryWrites=true&w=majority

In [ ]:
'''
Optimization suggestions from Kevin Chiv:

def make_vector(genres):
	# this could be done with either loops or mapping
	genre_vector = array of zeroes
	for genre in genres:
		pos = find genre's position in array
		genre_vector[pos] = 1
	return genre_vector
	
artist_genres_mapping = {
	'Beethoven': ['classical'],
	...
	'PSY': ['K-pop', 'hip-hop']
}
​
def make_weighted_average_vector(artists, weights):
	total_vector = [0, 0, ..., 0]
	for artist, weights in zip(artist, weights):
		artist_genres = artist_genres_mapping[artist]
		artist_vector = make_vector(artist_genres)
		total_vector += artist_vector * weight
	return total_vector/len(artists)
​
weighted_average_vector = make_weighted_average_vector(['Kayne', 'AC/DC'], weights = [1, 3])
​
artist_scores = {}
​
for artist, genres in artist_genres_mapping:
	artist_vector = make_vector(genres)
	score = cosine_similarity(weighted_average_vector, vector)
	artist_scores[artist] = score
​
artist_scores = sorted(artist_scores.items(), key = lambda x: x[1])
'''

In [124]:
#genres_clean[:10]

## Check-in w/ Don:

- use a double loop:
  - main loop (artist a): calculates adjacent artists for artist a
  - inner loop (artist b): for each of the other artists (artist b1 - bn), calculate their genre array one at a time and then do the pairwise cosine similarity between a and bn
  
- this will produce a list of similarities (cosine scores) between artist a and all the bs, most will be 0!
  - keep only the top n items in this list
  - construct a new array with the artist and the n closest artists
  
- this will be inefficient, and some of it can be vectorized (definitely the inner one)

In [5]:
def make_artist_vector(genres, artists):
    
    '''
    Inputs 
    - genres: list of genres by name
    - artists: dict with artists as keys and a list of their genres as values
    '''
    
    arr = np.zeros(len(genres), dtype=bool)
    seed_artist = input('What artist are you searching?')
    
    if seed_artist in artists:
        styles = artists[seed_artist]
        style_pos = []
        
        for idx,genre in enumerate(genres):
            if genre in styles:
                pos = idx
                style_pos.append(pos)
        
        np.put(arr, style_pos, 1)
        return arr, seed_artist
    
    else:
        print('Sorry, that artist does not exist in this dataset.')

In [83]:
test1a = make_artist_vector(genres_clean, artists)

What artist are you searching? Fleetwood Mac


In [8]:
#test[test==True]

array([False, False, False, ..., False, False, False])

In [84]:
test1a[1]

'Fleetwood Mac'

In [85]:
len(test1a[0])

5071

In [86]:
def compare_artist(genres, artists_dict, seed, seed_arr):
    
    similarities = []
    artist_names = []
    
    for artist in artists_dict:
        if artist != seed:
            artist_names.append(str(artist))
            comp_arr = np.zeros(len(genres), dtype=bool)
            styles = artists_dict[artist]
            style_pos = []
            
            for idx,genre in enumerate(genres):
                if genre in styles:
                    pos = idx
                    style_pos.append(pos)
            
            np.put(comp_arr, style_pos, 1)
            sim = cosine_similarity(seed_arr.reshape(1,-1), comp_arr.reshape(1,-1))
            similarities.append(sim)
        else:
            continue
    
    #return similarities
    artist_sims = []
    
    for a,artist in zip(similarities,artist_names):
        for b in a:
            for c in b:
                tup = (artist, c) 
                artist_sims.append(tup)       
    
    return artist_sims         

In [87]:
test1b = compare_artist(genres_clean, artists, test1a[1], test1a[0])

In [88]:
len(test1b)

395470

In [89]:
#test1b[1][0]

In [90]:
#cosine_sims_test = []

#for a in test2:
#    for b in a:
#        for c in b:
#            cosine_sims_test.append(c)

In [91]:
#len(cosine_sims_test)

In [92]:
#for i in range(len(test2)):
#    if test2[i][1] > 0:
#        print(test2[i])

In [93]:
def sort_tuple(tup): 
  
    #reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    tup.sort(key = lambda x: x[1], reverse=True) 
    return tup[:10] 

In [94]:
test1c = sort_tuple(test1b)

In [101]:
for i in test1c:
    print(i[0])

Daryl Hall & John Oates
TOTO
Dire Straits
Pat Benatar
Huey Lewis & The News
Kenny Loggins
Rick Springfield
Eagles
Foreigner
Heart


In [97]:
artists['Fleetwood Mac']

['album rock',
 'classic rock',
 'mellow gold',
 'rock',
 'soft rock',
 'yacht rock']

In [98]:
artists['TOTO']

['album rock',
 'classic rock',
 'mellow gold',
 'rock',
 'soft rock',
 'yacht rock']